# cuBLAS example with C++ kernel (no CUDA mode)

Notebook bases on NVIDIA CUDA example `simpleCUBLAS`

In [ ]:
#include <iostream>
#include <vector>
#include <numeric>

#include <cublas_v2.h>

#pragma cling(load "libcublas.so")

## Check functions

In [ ]:
inline void cuCheck(cudaError_t code) {
  if (code != cudaSuccess) {
    std::cerr << "Error code: " << code << std::endl
              << cudaGetErrorString(code) << std::endl;
  }
}

In [ ]:
inline void cuCheck(cublasStatus_t code) {
  if (code != CUBLAS_STATUS_SUCCESS) {
    std::cerr << "CUBLAS Error code: " << code << std::endl;
  }
}

## Initialize variables

In [ ]:
// initialize matrices
__global__ void init(float *matrix, int size){
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    if (x < size)
        matrix[x] = x;
}

In [ ]:
const int dim = 1024;
const int threads = 32;
const int blocks = ((dim*dim) + threads - 1) / threads;

// host memory
float *h_C;
// device memory
float *d_A = 0;
float *d_B = 0;
float *d_C = 0;

float alpha = 1.0f;
float beta = 0.0f;

cublasHandle_t handle;

In [ ]:
// allocate host memory
h_C = new float[dim * dim];

// allocate device memory
cuCheck(cudaMalloc((void **)&d_A, dim * dim * sizeof(d_A[0])));
cuCheck(cudaMalloc((void **)&d_B, dim * dim * sizeof(d_A[0])));
cuCheck(cudaMalloc((void **)&d_C, dim * dim * sizeof(d_A[0])));

cuCheck(cublasCreate(&handle));

In [ ]:
init<<<blocks, threads>>>(d_A, dim*dim);
cuCheck(cudaGetLastError());
init<<<blocks, threads>>>(d_B, dim*dim);
cuCheck(cudaGetLastError());

## Copy Memory and run cuBLAS

In [ ]:
cuCheck(cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, dim, dim, dim, &alpha, d_A, dim, d_B, dim, &beta, d_C, dim));

In [ ]:
// copy result back
cuCheck(cublasGetVector(dim*dim, sizeof(h_C[0]), d_C, 1, h_C, 1));

## Verify result

In [ ]:
float *h_A = new float[dim*dim]; 
float *h_B = new float[dim*dim]; 

cuCheck(cublasGetVector(dim*dim, sizeof(h_A[0]), d_A, 1, h_A, 1));
cuCheck(cublasGetVector(dim*dim, sizeof(h_B[0]), d_B, 1, h_B, 1));

float *h_C_ref = new float[dim*dim]; 
float error_norm = 0.f;
float ref_norm = 0.f;
float diff = 0.f;

In [ ]:
void simple_sgemm(int n, float alpha, const float *A, const float *B,
                         float beta, float *C)
{
    int i;
    int j;
    int k;

    for (i = 0; i < n; ++i)
    {
        for (j = 0; j < n; ++j)
        {
            float prod = 0;

            for (k = 0; k < n; ++k)
            {
                prod += A[k * n + i] * B[j * n + k];
            }

            C[j * n + i] = alpha * prod + beta * C[j * n + i];
        }
    }
}


In [ ]:
simple_sgemm(dim, alpha, h_A, h_B, beta, h_C_ref);

In [ ]:
for (int i = 0; i < dim*dim; ++i)
{
    diff = h_C_ref[i] - h_C[i];
    error_norm += diff * diff;
    ref_norm += h_C_ref[i] * h_C_ref[i];
}

error_norm = (float)sqrt((double)error_norm);
ref_norm = (float)sqrt((double)ref_norm);

if (fabs(ref_norm) < 1e-7)
    std::cerr << "reference norm is 0" << std::endl;

if (error_norm / ref_norm < 1e-6f)
    std::cout << "cuBLAS test passed" << std::endl;
else
    std::cout << "cuBLAS test failed" << std::endl;

## Clean up

In [ ]:
free(h_A);
free(h_B);
free(h_C);
free(h_C_ref);

cuCheck(cudaFree(d_A));
cuCheck(cudaFree(d_B));
cuCheck(cudaFree(d_C));

cublasDestroy(handle);